## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-02-18-58-49 +0000,nyt,Supreme Court Seems Skeptical of N.J. in Dispu...,https://www.nytimes.com/2025/12/02/us/politics...
1,2025-12-02-18-57-57 +0000,nyt,"Elon Musk’s Foundation Grows to $14 Billion, b...",https://www.nytimes.com/2025/12/02/us/politics...
2,2025-12-02-18-55-58 +0000,nyt,Putin and Witkoff Meet in Russia as U.S. Pushe...,https://www.nytimes.com/2025/12/02/world/europ...
3,2025-12-02-18-53-09 +0000,wapo,Former E.U. foreign policy chief detained in f...,https://www.washingtonpost.com/world/2025/12/0...
4,2025-12-02-18-49-13 +0000,nyt,Russian Launch Site Mishap Leaves Country’s Sp...,https://www.nytimes.com/2025/12/01/science/rus...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
74,trump,62
65,new,25
177,house,17
128,out,14
527,white,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
250,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...,138
307,2025-12-01-21-10-44 +0000,nypost,White House doesn’t rule out boots on the grou...,https://nypost.com/2025/12/01/us-news/trump-do...,130
237,2025-12-02-01-32-00 +0000,wsj,"Officially, the search for a new Federal Reser...",https://www.wsj.com/politics/policy/trump-fed-...,124
218,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,124
97,2025-12-02-14-41-48 +0000,nypost,Former Honduran President Juan Orlando Hernand...,https://nypost.com/2025/12/02/world-news/forme...,122


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
250,138,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...
108,65,2025-12-02-13-54-17 +0000,nypost,Pete Hegseth didn’t order killing of survivors...,https://nypost.com/2025/12/02/us-news/pete-heg...
97,60,2025-12-02-14-41-48 +0000,nypost,Former Honduran President Juan Orlando Hernand...,https://nypost.com/2025/12/02/world-news/forme...
77,55,2025-12-02-15-55-11 +0000,nypost,Vladimir Putin says he’s ready to go to war wi...,https://nypost.com/2025/12/02/world-news/putin...
68,45,2025-12-02-16-04-52 +0000,nypost,Samsung unveils first triple-folding phone ahe...,https://nypost.com/2025/12/02/business/samsung...
218,44,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...
103,42,2025-12-02-14-22-53 +0000,nypost,Mom of Texas cheerleader noticed ‘weird’ detai...,https://nypost.com/2025/12/02/us-news/mom-of-d...
307,40,2025-12-01-21-10-44 +0000,nypost,White House doesn’t rule out boots on the grou...,https://nypost.com/2025/12/01/us-news/trump-do...
179,37,2025-12-02-09-27-55 +0000,nypost,Pope Leo XIV prays at site of 2020 Beirut port...,https://nypost.com/2025/12/02/world-news/pope-...
226,35,2025-12-02-02-43-18 +0000,nypost,DHS chief Kristi Noem wants full travel ban on...,https://nypost.com/2025/12/01/us-news/dhs-chie...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
